<a href="https://colab.research.google.com/github/marcelounb/tensorflow_udacity/blob/master/02_fashion_mnist_CNNs_my_way.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from keras.datasets import fashion_mnist
import matplotlib.pyplot as plt
import numpy as np
import math
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import Dropout 
from keras.utils import np_utils
from keras.utils import to_categorical
from keras.optimizers import SGD

# Imports for the CNN
from keras.layers import Flatten 
from keras.layers.convolutional import Convolution2D 
from keras.layers.convolutional import MaxPooling2D
from keras import backend as K 
from random import randrange

In [2]:
import tensorflow as tf
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

# Loading Data (baixando data)

In [3]:
# load (downloaded if needed) the MNIST dataset 
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [4]:
print("Number of training examples: {}".format(len(X_train)))
print("Number of test examples:     {}".format(len(X_test)))
X_train.shape, y_train.shape ,X_test.shape, y_test.shape

Number of training examples: 60000
Number of test examples:     10000


((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [5]:
# reshape dataset to have a single channel
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))
print("Number of training examples: {}".format(len(X_train)))
print("Number of test examples:     {}".format(len(X_test)))
X_train.shape, y_train.shape ,X_test.shape, y_test.shape

Number of training examples: 60000
Number of test examples:     10000


((60000, 28, 28, 1), (60000,), (10000, 28, 28, 1), (10000,))

In [6]:
# one hot encode target values
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [7]:
X_train.shape, y_train.shape ,X_test.shape, y_test.shape

((60000, 28, 28, 1), (60000, 10), (10000, 28, 28, 1), (10000, 10))

In [8]:
# scale pixels
# convert from integers to floats
X_train_norm = X_train.astype('float32')
X_test_norm = X_test.astype('float32')
# normalize to range 0-1
X_train_norm = X_train_norm / 255.0
X_test_norm = X_test_norm / 255.0

# Building the model

In [9]:
def baseline_model(): 
  # create model 
  model = Sequential() 
  model.add(Flatten(input_shape=(28,28,1)))
  model.add(Dense(128, activation= 'relu' )) 
  model.add(Dense(10, activation= 'softmax' )) 

  # Compile model 
  model.compile(loss= 'sparse_categorical_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ]) 
  return model

In [10]:
def cnn_model():
  # create model 
  model = Sequential() 
  model.add(Convolution2D(32, (3, 3), input_shape=(28, 28, 1), activation= 'relu' )) 
  model.add(MaxPooling2D(pool_size=(2, 2), strides=2)) 
  model.add(Convolution2D(64, (3, 3), padding='same', activation='relu' )) 
  model.add(MaxPooling2D(pool_size=(2, 2), strides=2))  
  model.add(Flatten()) 
  model.add(Dense(128, activation= 'relu' )) 
  model.add(Dense(10, activation= 'softmax' ))

  # Compile model 
  model.compile(loss= 'sparse_categorical_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ]) 
  return model

In [20]:
def cnn_model_my_way():
  # create model 
  model = Sequential() 
  model.add(Convolution2D(32, (3, 3), kernel_initializer='he_uniform' ,input_shape=(28, 28, 1), activation= 'relu' )) 
  model.add(MaxPooling2D((2, 2)))
  model.add(Flatten())
  model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(10, activation='softmax'))


  # Compile model 
  model.compile(loss= 'categorical_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ]) 
  return model

In [17]:
def define_model():
	model = Sequential()
	model.add(Convolution2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='softmax'))
	# compile model
	opt = SGD(lr=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

# Fitting the model - My way around

In [18]:
# build the model 
model = define_model() 
# K.set_image_data_format('channels_first')
# Fit the model 
history = model.fit(X_train_norm, y_train, validation_data=(X_test_norm, y_test), epochs=10, batch_size=200, verbose=2)

Epoch 1/10
300/300 - 22s - loss: 0.5335 - accuracy: 0.8135 - val_loss: 0.4530 - val_accuracy: 0.8321
Epoch 2/10
300/300 - 22s - loss: 0.3712 - accuracy: 0.8703 - val_loss: 0.3704 - val_accuracy: 0.8701
Epoch 3/10
300/300 - 22s - loss: 0.3299 - accuracy: 0.8834 - val_loss: 0.3548 - val_accuracy: 0.8723
Epoch 4/10
300/300 - 22s - loss: 0.2984 - accuracy: 0.8943 - val_loss: 0.3194 - val_accuracy: 0.8844
Epoch 5/10
300/300 - 22s - loss: 0.2781 - accuracy: 0.8994 - val_loss: 0.3180 - val_accuracy: 0.8843
Epoch 6/10
300/300 - 22s - loss: 0.2607 - accuracy: 0.9066 - val_loss: 0.2911 - val_accuracy: 0.8938
Epoch 7/10
300/300 - 22s - loss: 0.2465 - accuracy: 0.9125 - val_loss: 0.2843 - val_accuracy: 0.8948
Epoch 8/10
300/300 - 22s - loss: 0.2313 - accuracy: 0.9172 - val_loss: 0.2768 - val_accuracy: 0.8984
Epoch 9/10
300/300 - 22s - loss: 0.2209 - accuracy: 0.9201 - val_loss: 0.2880 - val_accuracy: 0.8961
Epoch 10/10
300/300 - 22s - loss: 0.2125 - accuracy: 0.9240 - val_loss: 0.2683 - val_accura

In [21]:
# build the model 
model2 = cnn_model_my_way() 
# K.set_image_data_format('channels_first')
# Fit the model 
history2 = model2.fit(X_train_norm, y_train, validation_data=(X_test_norm, y_test), epochs=10, batch_size=200, verbose=2)

Epoch 1/10
300/300 - 22s - loss: 0.4497 - accuracy: 0.8435 - val_loss: 0.4422 - val_accuracy: 0.8513
Epoch 2/10
300/300 - 22s - loss: 0.2910 - accuracy: 0.8962 - val_loss: 0.2956 - val_accuracy: 0.8914
Epoch 3/10
300/300 - 22s - loss: 0.2466 - accuracy: 0.9116 - val_loss: 0.2821 - val_accuracy: 0.8978
Epoch 4/10
300/300 - 22s - loss: 0.2197 - accuracy: 0.9204 - val_loss: 0.2636 - val_accuracy: 0.9027
Epoch 5/10
300/300 - 22s - loss: 0.1964 - accuracy: 0.9292 - val_loss: 0.2603 - val_accuracy: 0.9059
Epoch 6/10
300/300 - 22s - loss: 0.1801 - accuracy: 0.9337 - val_loss: 0.2654 - val_accuracy: 0.9065
Epoch 7/10
300/300 - 22s - loss: 0.1645 - accuracy: 0.9401 - val_loss: 0.2609 - val_accuracy: 0.9068
Epoch 8/10
300/300 - 22s - loss: 0.1524 - accuracy: 0.9448 - val_loss: 0.2587 - val_accuracy: 0.9112
Epoch 9/10
300/300 - 22s - loss: 0.1396 - accuracy: 0.9488 - val_loss: 0.2634 - val_accuracy: 0.9091
Epoch 10/10
300/300 - 22s - loss: 0.1256 - accuracy: 0.9547 - val_loss: 0.2671 - val_accura